COST FUNCTION: MEAN SQUARED ERROR

OPTIMIZER: BFGS

input weight matrix is converted to a vector of weights between lunar stations (excluding self weights)

In [ ]:
import numpy as np
import random
import math
import scipy as sp

np.random.seed(123)

%run "./DiffusionLunarKF.ipynb"
%run "./CentralizedLunarKF.ipynb"
%run "./FilterComparison.ipynb"
%run "./VectorFormatting.ipynb"

[[0.00656212 0.00087641]
 [0.00087641 0.00015093]]
[[24.40369964  0.          0.          0.        ]
 [ 0.          0.20383392  0.          0.        ]
 [ 0.          0.         24.40369964  0.        ]
 [ 0.          0.          0.          0.20383392]]
[0.2808067537688888, 0.11536742677858898, 0.09146337207788346, 0.22228249836798888, 0.30138564230254933, 0.5874526275750482, 0.5332787972708654, 0.6243884898665294]
[[0.29007995 0.28080675 0.11536743 0.09146337 0.2222825 ]
 [0.69861436 0.30138564 0.         0.         0.        ]
 [0.41254737 0.         0.58745263 0.         0.        ]
 [0.4667212  0.         0.         0.5332788  0.        ]
 [0.37561151 0.         0.         0.         0.62438849]]
Row Sums:  [1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
def run_fixed_seed(C,seed, sats, iterations = 100):

    adj = np.array([[1,1,1,1,1],
                [1,1,0,0,0],
                [1,0,1,0,0],
                [1,0,0,1,0],
                [1,0,0,0,1]])
    D = 3*adj
    n = len(adj)

    np.random.seed(seed)
    true_biases = np.array([[np.random.normal(0,np.sqrt(12/(c**2))) for _ in range(n)]]).T
    true_drifts = np.array([[np.random.normal(0,np.sqrt(0.1/(c**2))) for _ in range(n)]]).T
    # true_drifts = np.array([[0 for _ in range(n)]]).T

    F = np.array([[1,dt],[0,1]])
    F_full = np.kron(np.eye(n),F)

    x = c*np.vstack(tuple([np.array([true_biases[i],true_drifts[i]]) for i in range(n)]))

    # random initial estimates for each node

    x0 = [np.array([[np.random.normal(0,np.sqrt(12))],[np.random.normal(0,np.sqrt(0.1))]]) for i in range(n)]
    x0_cf = np.vstack(tuple(x0))
    # x0 = [np.array([[0],[0]]) for _ in range(n)]

    P = [100*np.copy(R(1)) for _ in range(n)]
    P_prev = np.block([[P[i] if i==j else np.zeros((2,2)) for j in range(n)] for i in range(n)])

    stations = [Station(i) for i in range(n)]

    filter_initialize(stations,D,x0,P)

    Q_10x10 = np.kron(np.eye(n),Q)

    kf = KalmanFilter(A = F_full, H = H_cf, Q = Q_10x10, R = R_cf, P = P_prev, x0 = x0_cf)

    # num_msmts = np.random.randint(0,10,(iterations,5))
    num_msmts = np.array([sats for _ in range(iterations)])
    filter_outputs = run_both_filters(iterations, num_msmts,C,F_full,stations,kf, x)
    return filter_outputs

In [ ]:
def MSE(vec,C = None):
    ref = np.array([[1,1,1,1,1],
                [1,1,0,0,0],
                [1,0,1,0,0],
                [1,0,0,1,0],
                [1,0,0,0,1]])
    if C is None:
        C = convert_to_adj(vec,ref)

    n = len(C)
    iterations = 100
    seeds = [i for i in range(10)]
    avg = 0
    for i in range(10):
        outputs = run_fixed_seed(C,seeds[i],[3,0,3,3,3],iterations)

        errors_df = outputs[0]
        errors_cf = outputs[1]

        # squared_error_iteration = lambda i: sum([x**2 for x in errors_df[i,:,0,0]])
        squared_from_cf_iter = lambda i: sum([(errors_cf[i][2*stn][0] - errors_df[i,stn,0,0])**2 for stn in range(n)])
        avg += sum([x**2 for x in [squared_from_cf_iter(i) for i in range(iterations)]])/(n*iterations)

    return avg/10
    # squared_from_cf_iter = lambda i: sum([(predictions_cf[i][2*stn][0] - predictions_df[i,stn,0,0])**2 for stn in range(n)])
    # return sum([squared_from_cf_iter(i) for i in range(iterations)])/(n*iterations)

In [ ]:
#Run BFGS for LTT
def mse_bfgs(seed = 123):
  adj = np.array([[1,1,1,1,1],
                [1,1,0,0,0],
                [1,0,1,0,0],
                [1,0,0,1,0],
                [1,0,0,0,1]])
  #Linear Constraints:
  individual_ones = [[1 if i==j else 0 for j in range(8)] for i in range(8)]
  # others = [[1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  #           [0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  #           [0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],
  #           [0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0],
  #           [0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0],
  #           [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0],
  #           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0],
  #           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1]]
  others = [[1,1,1,1,0,0,0,0]]
  A = np.array(individual_ones+others)
  cons = sp.optimize.LinearConstraint(A, lb = np.zeros(9), ub = np.ones(9))
  bounds = sp.optimize.Bounds(lb = np.zeros(8), ub = np.ones(8))

  #Initial Guess:
  x0 = [0,0.225,0.225,0.225,0.55,0.9,0.9,0.9]

  result = sp.optimize.minimize(fun=MSE,x0=x0, bounds=bounds, constraints=cons)

  print("Optimized Weights Vector: ", np.around(result.x, 4))
  print("\nCorresponding Adjacency Matrix:")
  print(convert_to_adj(np.around(result.x, 4),adj))
  print("\nResulting MSE: ", round(result.fun, 4))

  return result

v = mse_bfgs()

Optimized Weights Vector:  [0.0012 0.3027 0.2018 0.2027 0.5942 0.9098 0.9079 0.9096]

Corresponding Adjacency Matrix:
[[0.2916 0.0012 0.3027 0.2018 0.2027]
 [0.4058 0.5942 0.     0.     0.    ]
 [0.0902 0.     0.9098 0.     0.    ]
 [0.0921 0.     0.     0.9079 0.    ]
 [0.0904 0.     0.     0.     0.9096]]

Resulting MSE:  139.4686


In [ ]:
#[6,1,6,6,6]


# v = [0.1883, 0.1506, 0.0708, 0.5925, 0.2922, 0.6023, 0.0667, 0.2585, 0.5671, 0.025,
#  0.2066, 0.0653, 0.091,  0.4996, 0.1046, 0.4709, 0.396,  0.1215, 0.046,  0.279 ]
v = [0, 0.1, 0.2, 0.2, 0.6, 0.9, 0.9, 0.9]
# adj = np.array([[1,1,0,1,1,0,0,0],
#                 [1,1,1,0,0,0,0,0],
#                 [0,1,1,1,0,0,1,0],
#                 [1,0,1,1,0,0,0,0],
#                 [1,0,0,0,1,1,0,1],
#                 [0,0,0,0,1,1,1,0],
#                 [0,0,1,0,0,1,1,1],
#                 [0,0,0,0,1,0,1,1]])

adj = np.array([[1,1,1,1,1],
                [1,1,0,0,0],
                [1,0,1,0,0],
                [1,0,0,1,0],
                [1,0,0,0,1]])

print(" Weight Vector: ", v)
print(convert_to_adj(v,adj))
print("Resulting MSE: ",MSE(v))

 Weight Vector:  [0, 0.1, 0.2, 0.2, 0.6, 0.9, 0.9, 0.9]
[[0.5 0.  0.1 0.2 0.2]
 [0.4 0.6 0.  0.  0. ]
 [0.1 0.  0.9 0.  0. ]
 [0.1 0.  0.  0.9 0. ]
 [0.1 0.  0.  0.  0.9]]
Resulting MSE:  130.8971145549425
